<a href="https://colab.research.google.com/github/kmkarakaya/Deep-Learning-Tutorials/blob/master/RaporHaz%C4%B1rlay%C4%B1c%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Murat Karakaya Akademi.

Youtube'dan bu içeriği seyredin: https://youtube.com/live/cTIzVReqSBY

This notebook provides a quick example of how to ask several questions at once to a pdf file using Gemini 1.5 Flash API.

In [1]:
!pip install -U -q google-generativeai

In [2]:
import google.generativeai as genai

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [3]:
from google.colab import userdata
google_api_key=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=google_api_key)

Upload the pdf file.

In [7]:
# select and upload a file from local directory
from google.colab import files
uploaded = files.upload()

Saving 3240730_Proje_Onerisi.pdf to 3240730_Proje_Onerisi.pdf


In [8]:
uploaded_keys = uploaded.keys()  # Get the dict_keys object
file_name = list(uploaded_keys)[0]  # Convert to a list and get the first element
print(file_name)

3240730_Proje_Onerisi.pdf


Convert the pdf file to a text file

In [4]:
!apt-get update && apt-get install -y poppler-utils


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 3s (80.1 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
0 upgraded, 0 newly i

In [10]:
!pdftotext "$file_name" text_file.txt

Upload the text file to the Generative AI API so that we can include it in a chat or prompt.

In [5]:
"""Uploads a file to the Generative AI API.

  Returns:
    A `google.generativeai.v1beta2.types.GcsSource` object representing the uploaded file.
  """
def upload_file():
  print(f"Uploading file...")
  text_file = genai.upload_file(path="text_file.txt")
  print(f"Completed upload: {text_file.uri}")
  return text_file



In [ ]:
text_file = upload_file()

## Generate Content

After the file has been uploaded, we can make `GenerateContent` or `Chat` requests that reference the File API URI. Then we will ask the model several questions regarding the content of the pdf file.

In [6]:
from IPython.display import Markdown
from IPython.display import display
import textwrap

In [7]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
chat_session = None
def build_model(text_file):
  global chat_session
  generation_config = {
    "temperature": 0.2,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
  }

  model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
    # safety_settings = Adjust safety settings
    # See https://ai.google.dev/gemini-api/docs/safety-settings
    system_instruction="""Yüklenen belgedeki bilgilere göre Türkçe cevap ver.
    Eğer sorunun cevabı belgede bulunmuyorsa 'Belgede Cevap Bulunmuyor' yaz.
    """,
  )

  chat_session = model.start_chat( history=[]
  )

  response = chat_session.send_message(["Yüklenen belgeyi bir cümle ile özetle", text_file])

  display(to_markdown(response.text))



In [15]:
build_model(text_file)

> Bu belge, TÜBİTAK'a sunulan, "Biomarkerlara Entegre olarak Veri Toplayabilen, Derin Öğrenme ile Akıl Sağlığının İnsan Bedenine Etkisini Ölçümleme ve Takip Platformu" adlı bir Ar-Ge projesinin ayrıntılı bir değerlendirme ve izleme raporudur. 


In [9]:
def chat(prompt):
  try:
    response = chat_session.send_message(prompt)
    markdown_text = to_markdown(response.text)
    # If in a Jupyter notebook, you can display the markdown using display()
    display(markdown_text)
    return response.text
  except ValueError:
    # If the response doesn't contain text, check if the prompt was blocked.
    print(response.prompt_feedback)
    # Also check the finish reason to see if the response was blocked.
    print(response.candidates[0].finish_reason)
    # If the finish reason was SAFETY, the safety ratings have more details.
    print(response.candidates[0].safety_ratings)
  except Exception as e:
      print("An unexpected error occurred:", e)



In [10]:
questions=["Makalenin yazarları kimdir?",
           "Hangi modeller kullanılmıştır?",
           "Kaç referans verilmiştir?",
           "Kaç yılında basılmıştır"]

In [11]:
def generate_report(questions):
  report_text = ""
  report_text += f"\n## SORULARINIZ VE CEVAPLARI\n"
  for question in questions:
    report_text += f"\n## {question}\n"
    answer = chat(question)
    report_text += f"\n{answer}\n"
  return  report_text



In [ ]:
report_text = generate_report(questions)

In [19]:
# display the Q&A response
display(to_markdown(report_text))

> 
> ## SORULARINIZ VE CEVAPLARI
> 
> ## Makalenin yazarları kimdir?
> 
> Belgede cevap bulunmuyor. 
> 
> 
> ## Hangi modeller kullanılmıştır?
> 
> Belgede, derin öğrenme modelleri kullanılacağı belirtiliyor. Özellikle CNN (Convolutional Neural Network) ve LSTM (Long Short-Term Memory) modellerinin birlikte kullanılacağı belirtiliyor. Ayrıca, DBSCAN, Isolation Forest, Advance Statistical Methods gibi algoritmalar da kullanılacağı belirtiliyor. 
> 
> 
> ## Kaç referans verilmiştir?
> 
> Belgede, 10 referans verilmiştir. 
> 
> 
> ## Kaç yılında basılmıştır
> 
> Belgede basım tarihi bulunmuyor. 
> 


In [12]:
# Install the required libraries
!pip install python-docx markdown html2text beautifulsoup4
# Import necessary libraries
import markdown
from docx import Document
from google.colab import files

In [13]:
from docx import Document
from bs4 import BeautifulSoup
from google.colab import files

# Function to convert the Q&A in Markdown to HTML
def convert_Markdown_to_HTML(report_text):
  html_text = markdown.markdown(report_text)
  return html_text

# Function to add the Q&A in HTML to a Word document
def add_html_to_word(html_text, doc):
    soup = BeautifulSoup(html_text, 'html.parser')

    for element in soup:
        if element.name == 'h1':
            doc.add_heading(element.get_text(), level=1)
        elif element.name == 'h2':
            doc.add_heading(element.get_text(), level=2)
        elif element.name == 'h3':
            doc.add_heading(element.get_text(), level=3)
        elif element.name == 'h4':
            doc.add_heading(element.get_text(), level=4)
        elif element.name == 'h5':
            doc.add_heading(element.get_text(), level=5)
        elif element.name == 'h6':
            doc.add_heading(element.get_text(), level=6)
        elif element.name == 'p':
            doc.add_paragraph(element.get_text())
        elif element.name == 'ul':
            for li in element.find_all('li'):
                doc.add_paragraph(li.get_text(), style='List Bullet')
        elif element.name == 'ol':
            for li in element.find_all('li'):
                doc.add_paragraph(li.get_text(), style='List Number')
        elif element.name:
            doc.add_paragraph(element.get_text())  # For any other tags



In [ ]:
# Create a Word document
doc = Document()

# Convert Markdown to HTML
html_text = convert_Markdown_to_HTML(report_text)

# Add the HTML content to the Word document
add_html_to_word(html_text, doc)

# Save the document
doc_name = file_name.replace(".pdf", ".docx")
doc_name = "Rapor " + doc_name
doc.save(doc_name)

# Print a message indicating that the document has been generated
print("Document generated successfully!")

In [22]:
# Download the document
files.download(doc_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Delete Files

Files are automatically deleted after 2 days or you can manually delete them using `files.delete()`.

In [23]:
genai.delete_file(text_file.name)

In [14]:
#delete the file_name file
!rm "$file_name"

#delete the doc_name file
!rm "$doc_name"

#delete the text_file.txt
!rm "text_file.txt"

rm: cannot remove '': No such file or directory
rm: cannot remove '': No such file or directory


## CREATE AN INTERFACE WITH GRADIO

The File API accepts files under 2GB in size and can store up to 20GB of files per project. Learn more about the [File API](https://github.com/google-gemini/cookbook/blob/main/quickstarts/File_API.ipynb) here.

In [15]:
!pip install gradio

In [18]:
import gradio as gr
#import pdfplumber
import io
import shutil


def process_pdf(pdf_file, user_questions):
    # pdf file name:
    file_name = pdf_file.split('/')[-1]
    print(file_name)

    # Save the uploaded file to the Colab content directory
    saved_file_path = "/content/" + file_name
    shutil.copyfile(pdf_file, saved_file_path)

    # Convert PDF to text
    '''
    pdf_bytes = io.BytesIO(pdf_file)
    with pdfplumber.open(pdf_bytes) as pdf:
        text = ''.join(page.extract_text() for page in pdf.pages)
    '''

    # Save text to a .txt file
    !pdftotext "$file_name" text_file.txt
    text_file = None

    '''
    # Convert PDF to text
    with pdfplumber.open(pdf_file) as pdf:
        text = ''.join(page.extract_text() for page in pdf.pages)
    with open("text_file.txt", "w", encoding="utf-8") as text_file:
        text_file.write(text)
    '''
    text_file=upload_file()
    build_model(text_file)
    print("Model built")

    # Split the user_questions by new lines to get a list of questions
    questions = user_questions.strip().split('\n')
    print(questions)
    report_text=generate_report(questions)
    print(report_text)

    #markdown_output = markdown.markdown(report_text)


    # Create a Word document
    doc = Document()

    # Convert Markdown to HTML
    html_text = convert_Markdown_to_HTML(report_text)

    # Add the HTML content to the Word document
    add_html_to_word(html_text, doc)

    # Save the document
    doc_name = file_name.replace(".pdf", ".docx")
    doc_name = "Rapor " + doc_name
    doc.save(doc_name)

    # Print a message indicating that the document has been generated
    print("Document generated successfully!")

    return html_text, doc_name

# Convert the list of questions into a single string with each question on a new line
questions_str = "\n".join(questions)

iface = gr.Interface(
    fn=process_pdf,
    inputs=[
        gr.File(label="Upload PDF", type="filepath"),
        gr.TextArea(label="Enter Questions",
                    placeholder="Type your questions here, one per line.",
                    value=questions_str)
    ],
    outputs=[
        gr.HTML(label="HTML Formatted Report"),
        gr.File(label="DOCX File Output", type="binary")
    ],
    title="REPORT GENERATOR: ASK YOUR QUESTIONS TO A PDF FILE BY MURAT KARAKAYA AKADEMI",
    description="Upload a PDF to ask questions and get the answers."
)

iface.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://cf1d6f7e7a890ea9b9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


64 Yazýlým Mühendisliði Eðitiminde Ders Öðrenme Çýktýlarýnýn Ölçümlenmesi.pdf
Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/2d5kmjm5aio2


> Bu çalışma, Atılım Üniversitesi Yazılım Mühendisliği bölümünde ders çıktılarının ölçülmesinde kullanılan iki farklı yöntemin karşılaştırılmasını ve MÜDEK akreditasyon sisteminde ders çıktısı ölçekleme çalışmaları için tavsiyeler sunmaktadır. 


Model built
['Makalenin yazarları kimdir?', 'Hangi modeller kullanılmıştır?', 'Kaç referans verilmiştir?', 'Kaç yılında basılmıştır?', 'Ana sonuçlar nelerdir?', 'Hangi dergide yada konferansta basılmıştır?']


> Makalenin yazarları Murat Karakaya, Çiğdem Turhan ve Ali Yazıcı'dır. 


...

> Makalede 12 referans verilmiştir. 


> Makalenin basım yılı belgede belirtilmemiş. 


> Makalenin ana sonuçları şunlardır:
> 
> * Öğrencilerin ders çıktılarını öğrendikleri konusundaki algıları, aldıkları notlara göre daha yüksek seviyededir.
> * Doğrudan ve dolaylı değerlendirmede ders çıktılarında görülen farklılıkların bir nedeni de notlandırmada sıklıkla kullanılan çan eğrisidir.
> * Öğretmen ve öğrenci kitlesindeki değişimin ders çıktılarının başarımını etkileme olasılığı vardır.
> * Yalnızca 7 dönemlik verilere dayanarak ders çıktılarının başarımına göre ders içeriklerinin ve işlenme yöntemlerinin iyileştirilmesinin sağlıklı sonuç vermeyeceği değerlendirilmektedir.
> * İleride yapılacak çalışmalarda ders çıktılarının ölçümlenmesiyle bu ölçümlerin ampirik verilerle desteklenmesi önem arz etmektedir.
> 
> Bu sonuçlar, MÜDEK kapsamında ders çıktılarının ölçümü için yeni metodların geliştirilmesi ve değerlendirilmesi gerektiğini göstermektedir. 


> Belgede makaleyi yayınlayan dergi veya konferans bilgisi bulunmuyor. 



## SORULARINIZ VE CEVAPLARI

## Makalenin yazarları kimdir?

Makalenin yazarları Murat Karakaya, Çiğdem Turhan ve Ali Yazıcı'dır. 


## Hangi modeller kullanılmıştır?

Makalede doğrudan ve dolaylı ölçüm tekniklerine dayalı iki farklı model kullanılmıştır. Doğrudan ölçüm, öğrencilerin sınav, proje, ödev notları gibi somut verileri içerirken, dolaylı ölçümde anket gibi yöntemler kullanılmıştır. 


## Kaç referans verilmiştir?

Makalede 12 referans verilmiştir. 


## Kaç yılında basılmıştır?

Makalenin basım yılı belgede belirtilmemiş. 


## Ana sonuçlar nelerdir?

Makalenin ana sonuçları şunlardır:

* Öğrencilerin ders çıktılarını öğrendikleri konusundaki algıları, aldıkları notlara göre daha yüksek seviyededir.
* Doğrudan ve dolaylı değerlendirmede ders çıktılarında görülen farklılıkların bir nedeni de notlandırmada sıklıkla kullanılan çan eğrisidir.
* Öğretmen ve öğrenci kitlesindeki değişimin ders çıktılarının başarımını etkileme olasılığı vardır.
* Yalnızca 7 dönemlik verilere dayan